In [1]:
# Load in relevant libraries
import pandas as pd
import numpy as np

In [2]:
# Read in the medical claims and pharmacy claims data
medical_claims = pd.read_csv('Data/med_full_final_melted.csv')
medical_claims = medical_claims.drop('Unnamed: 0', axis =1)

rx_claims = pd.read_csv('Data/rx_full_final.csv')
rx_claims = rx_claims.drop('Unnamed: 0', axis=1)

C:\Users\mpguo\AppData\Local\Temp\ipykernel_8504\449010284.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  rx_claims = pd.read_csv('Data/rx_full_final.csv')


In [3]:
# Get the unique Patient Life IDs from the medical claims data
unique_patient_ids = medical_claims['Member Life ID'].unique()
unique_patient_ids.size

99999

In [4]:
# Create a new dataframe to hold input data for logistic regressions
input_data = pd.DataFrame({'Member Life ID':unique_patient_ids, 'Target Diagnosis Indicator':0})

input_data.head(10)

,Member Life ID,Target Diagnosis Indicator
0,109514,0
1,11824943,0
2,2573644,0
3,2602510,0
4,32741106,0
5,32735876,0
6,3360996,0
7,2581913,0
8,2768555,0
9,2802661,0


In [5]:
# Define the other Base Codes that should be included as variables
base_codes = np.unique(medical_claims['Base Code'])

In [6]:
# Adding one-hot encoded indicators of possibly related diagnoses
for base_code in base_codes:
    input_data[base_code] = 0

for base_code in base_codes:
    current_slice = medical_claims.loc[medical_claims['Base Code'] == base_code]

    current_diag_ids = current_slice['Member Life ID'].unique()

    input_data[base_code].loc[input_data['Member Life ID'].isin(current_diag_ids)] = 1

input_data

C:\Users\mpguo\AppData\Local\Temp\ipykernel_8504\1333298232.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_data[base_code] = 0
C:\Users\mpguo\AppData\Local\Temp\ipykernel_8504\1333298232.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_data[base_code] = 0
C:\Users\mpguo\AppData\Local\Temp\ipykernel_8504\1333298232.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

,Member Life ID,Target Diagnosis Indicator,100,123,130,160,230,240,250,300,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
0,109514,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11824943,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2573644,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,2602510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,32741106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,455440,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
99995,226218,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99996,24552166,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99997,18835833,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Left join in the gender data from the medical claims dataframe
gender_data = medical_claims[['Member Life ID', 'Biological Gender']].drop_duplicates()
input_data = input_data.merge(gender_data, how = 'left', left_on = 'Member Life ID', right_on = 'Member Life ID')
input_data = input_data.drop_duplicates()

input_data = input_data.reset_index()
input_data = input_data.drop('index', axis = 1)

In [8]:
# Get the earliest Line Service From Date
line_service_data = medical_claims[['Member Life ID', 'Line Service From Date']]

line_service_data = line_service_data.groupby(['Member Life ID']).min()

line_service_data

,Line Service From Date
Member Life ID,
154,2017-01-26
169,2017-05-02
234,2017-08-25
383,2017-01-18
428,2017-02-14
...,...
38509136,2017-04-17
38511568,2018-12-28
38541263,2017-11-07


In [9]:
# Get the earliest Header Service From Date
header_service_data = medical_claims[['Member Life ID', 'Header Service From Date']]

header_service_data = header_service_data.groupby(['Member Life ID']).min()

header_service_data

,Header Service From Date
Member Life ID,
154,2017-01-26
169,2017-05-02
234,2017-08-25
383,2017-01-18
428,2017-02-14
...,...
38509136,2017-04-17
38511568,2018-12-28
38541263,2017-11-07


In [10]:
# Left join in the Line Service From Date medical claims dataframe
input_data = input_data.merge(line_service_data, how = 'left', left_on = 'Member Life ID', right_on = 'Member Life ID')
input_data = input_data.drop_duplicates()

input_data = input_data.reset_index()
input_data = input_data.drop('index', axis = 1)

# Left join in the Headaer Service From Date data from the medical claims dataframe
input_data = input_data.merge(header_service_data, how = 'left', left_on = 'Member Life ID', right_on = 'Member Life ID')
input_data = input_data.drop_duplicates()

input_data = input_data.reset_index()
input_data = input_data.drop('index', axis = 1)

In [11]:
# Left join in the Date of Birth data from the pharmacy dataframe
rx_birthdates = rx_claims[['Member Life ID', 'Birth Date']]
rx_birthdates = rx_birthdates.drop_duplicates()
rx_birthdates

# Left join in the gender data from the medical claims dataframe
input_data = input_data.merge(rx_birthdates, how = 'left', left_on = 'Member Life ID', right_on = 'Member Life ID')
input_data = input_data.drop_duplicates()

input_data = input_data.reset_index()
input_data = input_data.drop('index', axis = 1)

In [12]:
input_data = input_data.drop('Target Diagnosis Indicator', axis=1)

In [13]:
input_data

,Member Life ID,100,123,130,160,230,240,250,300,367,...,Z94,Z95,Z96,Z97,Z98,Z99,Biological Gender,Line Service From Date,Header Service From Date,Birth Date
0,109514,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,F,2017-03-02,2017-03-02,1984-12-18
1,11824943,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,F,2017-04-06,2017-04-06,NaN
2,2573644,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,F,2017-01-03,2017-01-03,NaN
3,2602510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,F,2017-02-09,2017-02-09,1959-12-04
4,32741106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,F,2017-01-04,2017-01-04,1990-11-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100424,455440,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,F,2017-04-21,2017-04-21,NaN
100425,226218,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,M,2017-02-16,2017-02-16,1943-06-03
100426,24552166,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,M,2018-01-30,2018-01-30,1989-12-30
100427,18835833,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,M,2017-10-05,2017-10-05,1955-08-26


In [14]:
input_data.to_csv('formatted_data_all_codes.csv')